#Configuration Parameters

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import sys
import os
import argparse
import csv

import imageio
import numpy as np
import pandas as pd

from keras.utils import multi_gpu_model, to_categorical
from keras.layers import Activation,Input
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
from tensorflow import convert_to_tensor
from tensorflow.keras.metrics import MeanIoU

sys.path.append("/content/drive/My Drive/tf-keras-SegNet")
from model import segnet
sys.path.append('/content/drive/My Drive')
from lasero.utils import training

Using TensorFlow backend.


In [0]:
base_dir = "/content/drive/My Drive/"

In [0]:
def get_image_list(metadata_file_path):
    with open(metadata_file_path, 'r') as f:
        return [i for i in csv.reader(f) if i]

In [0]:
metadata_path = os.path.join(base_dir, "Metadata")
dataset = "one_percent"
training_set_path = os.path.join(metadata_path, f'{dataset}_train.csv')
val_set_path = os.path.join(metadata_path, f'{dataset}_val.csv')

bands = [1, 2, 3] # number of bands

# if 'none', transfer weights cells should be skipped
weights = 'none'

num_classes = 2
class_weights = {0: 1.,
                 1: 10.}

loss_function = 'categorical_crossentropy'
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

batch_size = 4
epochs = 50
iou = MeanIoU(num_classes)
metrics = ['accuracy', iou]
stop_monitor_metric = "val_loss"

In [0]:
lr_monitor_metric = 'val_loss'
factor = 0.5
lr_patience = 1
min_lr = 0.00001

reduce_lr = ReduceLROnPlateau(monitor=lr_monitor_metric,
                              factor=factor,
                              patience=lr_patience,
                              verbose=1,
                              mode='auto',
                              min_lr=min_lr)

stop_monitor_metric = 'val_loss'
stop_patience = 9

early_stopper = EarlyStopping(monitor=stop_monitor_metric,
                              patience=stop_patience,
                              verbose=1,
                              mode='auto')

In [7]:
session_manager = training.SessionManager(base_dir)
input_component = session_manager.build_input_components(dataset, bands, weights)

opt_component = session_manager.build_opt_components(optimizer.__class__.__name__, loss_function, learning_rate)

training_components = session_manager.build_training_components(batch_size, epochs, class_weight)
lr_monitor_components = session_manager.build_lr_monitor_components(lr_monitor_metric, factor, lr_patience, min_lr)
stop_monitor_components = session_manager.build_stop_components(stop_monitor_metric, stop_patience)

model_prefix = training_components + lr_monitor_components + stop_monitor_components

session_path = os.path.join(input_component, opt_component, model_prefix)
print(session_path)

dataset=one_percent_bands=0-1-2-3-4-5-6-7-8-9-_weights=none/opt=SGD_loss=categorical_crossentropy_lr=0.001/batch=4_epochs=50_lrmonitor=val_loss_factor=0.5_patience=1_min=1e-05_stopmonitor=val_loss_patience=9


In [8]:
session_paths = training.create_session_paths(session_path)
for i in session_paths:
    print(session_paths[i])

/content/drive/My Drive/models/dataset=one_percent_bands=0-1-2-3-4-5-6-7-8-9-_weights=none/opt=SGD_loss=categorical_crossentropy_lr=0.001/batch=4_epochs=50_lrmonitor=val_loss_factor=0.5_patience=1_min=1e-05_stopmonitor=val_loss_patience=9
/content/drive/My Drive/models/dataset=one_percent_bands=0-1-2-3-4-5-6-7-8-9-_weights=none/opt=SGD_loss=categorical_crossentropy_lr=0.001/batch=4_epochs=50_lrmonitor=val_loss_factor=0.5_patience=1_min=1e-05_stopmonitor=val_loss_patience=9/history.json
/content/drive/My Drive/models/dataset=one_percent_bands=0-1-2-3-4-5-6-7-8-9-_weights=none/opt=SGD_loss=categorical_crossentropy_lr=0.001/batch=4_epochs=50_lrmonitor=val_loss_factor=0.5_patience=1_min=1e-05_stopmonitor=val_loss_patience=9/logs.csv
/content/drive/My Drive/models/dataset=one_percent_bands=0-1-2-3-4-5-6-7-8-9-_weights=none/opt=SGD_loss=categorical_crossentropy_lr=0.001/batch=4_epochs=50_lrmonitor=val_loss_factor=0.5_patience=1_min=1e-05_stopmonitor=val_loss_patience=9/config.json
/content/d

In [0]:
training_set_list = training.get_image_list(training_set_path)
val_set_list = training.get_image_list(val_set_path)
training_set_size = len(training_set_list)
val_set_size = len(val_set_list)

image_shape = (512, 512, len(bands))

# Transfer weights of matching layers from image-net trained vgg16 to segnet

In [10]:
segnet_model = segnet(image_shape, num_classes)
if weights != 'none':
    vgg_model = VGG16(include_top=False, weights=weights, input_shape=image_shape, classes=num_classes)
    seg_layer_names = [i for i in segnet_model.layers if "conv" in i.name]
    vgg_layer_names = [i for i in vgg_model.layers if "conv" in i.name]
    
    transferable_layer_names = {}
    for i in range(len(vgg_layer_names)):
        transferable_layer_names[seg_layer_names[i].name] = vgg_layer_names[i]
    
    layer_count = 0
    for i in segnet_model.layers:
        try:
            i.set_weights(transferable_layer_names[i.name].get_weights())
            layer_count += 1
        except KeyError:
            pass
    
    print(layer_count)

Build enceder done..
Build decoder done..


# Custom Generator

In [0]:
def data_gen(metadata_file_path, bands, batch_size):
    image_list = np.asarray(get_image_list(metadata_file_path))
    np.random.seed(1)
    np.random.shuffle(image_list)

    band_normalization_map = []
    counter = 0

    total_steps = image_list.shape[0] // batch_size
    while True:
        step_start = counter * batch_size
        step_end = step_start + batch_size
        images = []
        masks = []
        for j in range(step_start, step_end):
            images.append(np.load(image_list[j, 0])[:,:,bands])
            masks.append(np.load(image_list[j, 1]))

        y = to_categorical(np.array(masks), 2)
        yield np.array(images) / 65535, y.reshape((batch_size, y.shape[1] * y.shape[2], y.shape[3]))

        counter +=1

        if counter >= total_steps:
            counter = 0
            np.random.shuffle(image_list)

In [0]:
train_df = pd.read_csv(training_set_path)

#Training model 


In [13]:
train_data = data_gen(training_set_path, bands, batch_size)
val_data = data_gen(val_set_path, bands, batch_size)

segnet_model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

accuracy_checkpoint = ModelCheckpoint(os.path.join(session_paths['session_dir'], 'val_accuracy.h5'),
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')

loss_checkpoint = ModelCheckpoint(os.path.join(session_paths['session_dir'], 'val_loss.h5'),
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')

iou_checkpoint = ModelCheckpoint(os.path.join(session_paths['session_dir'], 'MeanIoU.h5'),
                             monitor='val_' + segnet_model.metrics[1].name,
                             verbose=1,
                             save_best_only=True,
                             mode='max')

# concatenation of save path is needed here to save model name dynamically by epoch
checkpoint = ModelCheckpoint(session_paths['session_dir'] + "/epoch{epoch:02d}.h5",
                             period=5,
                             save_weights_only=False,
                             save_best_only=False)

csv_logger = CSVLogger(session_paths["logs"])

callbacks_list = [iou_checkpoint, accuracy_checkpoint, loss_checkpoint, checkpoint, reduce_lr, csv_logger]

hist = segnet_model.fit(train_data,
                        steps_per_epoch=training_set_size // batch_size,
                        epochs=epochs,
                        validation_data=val_data,
                        validation_steps=val_set_size // batch_size,
                        verbose=1,
                        callbacks=callbacks_list,
                        class_weight=class_weight)

Epoch 1/50
192/192 [==============================] - 243s 1s/step - loss: 0.7639 - accuracy: 0.6073 - mean_io_u: 0.2500 - val_loss: 0.4904 - val_accuracy: 0.9046 - val_mean_io_u: 0.2500

Epoch 00001: val_mean_io_u improved from -inf to 0.25000, saving model to /content/drive/My Drive/models/dataset=one_percent_bands=0-1-2-3-4-5-6-7-8-9-_weights=none/opt=SGD_loss=categorical_crossentropy_lr=0.001/batch=4_epochs=50_lrmonitor=val_loss_factor=0.5_patience=1_min=1e-05_stopmonitor=val_loss_patience=9/MeanIoU.h5

Epoch 00001: val_accuracy improved from -inf to 0.90463, saving model to /content/drive/My Drive/models/dataset=one_percent_bands=0-1-2-3-4-5-6-7-8-9-_weights=none/opt=SGD_loss=categorical_crossentropy_lr=0.001/batch=4_epochs=50_lrmonitor=val_loss_factor=0.5_patience=1_min=1e-05_stopmonitor=val_loss_patience=9/val_accuracy.h5

Epoch 00001: val_loss improved from inf to 0.49041, saving model to /content/drive/My Drive/models/dataset=one_percent_bands=0-1-2-3-4-5-6-7-8-9-_weights=none